In [1]:
import sys
import os
import torch

sys.path.append(os.path.abspath(sys.path[0] + '/../../'))
__package__ = "deep_view_syn.notebook"
torch.cuda.set_device(2)
print("Set CUDA:%d as current device." % torch.cuda.current_device())
torch.autograd.set_grad_enabled(False)

from ..data.spherical_view_syn import *
from ..configs.spherical_view_syn import SphericalViewSynConfig
from ..my import netio
from ..my import util
from ..my import device
from ..my import view
from ..my.gen_final import GenFinal


def load_net(path):
    config = SphericalViewSynConfig()
    config.from_id(path[:-4])
    config.SAMPLE_PARAMS['perturb_sample'] = False
    #config.print()
    net = config.create_net().to(device.GetDevice())
    netio.LoadNet(path, net)
    return net


def find_file(prefix):
    for path in os.listdir():
        if path.startswith(prefix + '@'):
            return path
    return None


def load_views(data_desc_file) -> view.Trans:
    with open(data_desc_file, 'r', encoding='utf-8') as file:
        data_desc = json.loads(file.read())
        view_centers = torch.tensor(
            data_desc['view_centers'], device=device.GetDevice()).view(-1, 3)
        view_rots = torch.tensor(
            data_desc['view_rots'], device=device.GetDevice()).view(-1, 3, 3)
        return view.Trans(view_centers, view_rots)

scenes = {
    'gas': '__0_user_study/us_gas_all_in_one',
    'mc': '__0_user_study/us_mc_all_in_one',
    'bedroom': 'bedroom_all_in_one',
    'gallery': 'gallery_all_in_one',
    'lobby': 'lobby_all_in_one'
}

fov_list = [20, 45, 110]
res_list = [(128, 128), (256, 256), (256, 230)]
res_full = (1600, 1440)

Set CUDA:2 as current device.


In [2]:
scene = 'mc'
os.chdir(sys.path[0] + '/../data/' + scenes[scene])
print('Change working directory to ', os.getcwd())

fovea_net = load_net(find_file('fovea'))
periph_net = load_net(find_file('periph'))

# Load Dataset
views = load_views('nerf_views.json')
print('Dataset loaded.')
print('views:', views.size())
gen = GenFinal(fov_list, res_list, res_full, fovea_net, periph_net,
               device=device.GetDevice())

Change working directory to  /home/dengnc/deep_view_syn/data/__0_user_study/us_mc_all_in_one
Load net from fovea@nmsl-rgb_e10_fc128x4_d1-50_s32.pth ...
Load net from periph@nnmsl-rgb_e10_fc64x4_d1-50_s16.pth ...
Dataset loaded.
views: [110]


In [4]:
for view_idx in range(20):
    center = (0, 0)
    test_view = views.get(view_idx)
    images = gen.gen(center, test_view,
                     mono_trans=view.Trans(test_view.trans_point(
                         torch.tensor([-0.03, 0, 0], device=device.GetDevice())
                     ), test_view.r))
    #left_images = gen(center, view.Trans(
    #    test_view.trans_point(
    #        torch.tensor([-0.03, 0, 0], device=device.GetDevice())
    #    ), test_view.r), mono_trans=test_view, ret_raw=True)
    #right_images = gen(center, view.Trans(
    #    test_view.trans_point(
    #        torch.tensor([0.03, 0, 0], device=device.GetDevice())
    #    ), test_view.r), mono_trans=test_view, ret_raw=True)
    #plot_figures(images, center)

    outputdir = '/home/dengnc/deep_view_syn/data/__1_eval/output_mono_periph/ref_as_right_eye/%s/' % scene
    util.CreateDirIfNeed(outputdir)
    #for key in images:
    key = 'blended'
    util.WriteImageTensor(images[key], outputdir + 'view%04d_%s.png' % (view_idx, key))


In [5]:
import numpy as np
gaze_idx = 0
for y in np.linspace(-200, 200, 11):
    for x in np.linspace(-200, 200, 11):
        center = (int(x), int(y))
        test_view = views.get(0)
        images = gen.gen(center, test_view, True)
        #plot_figures(images, center)

        util.CreateDirIfNeed('output/eval_gaze')
        out_path = 'output/eval_gaze/gaze%03d_%d,%d.png' % (gaze_idx, x, y)
        util.WriteImageTensor(images['blended'], out_path)
        print('Output ' + out_path)
        gaze_idx += 1

TypeError: gen() takes 3 positional arguments but 4 were given